## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns={'EIN', 'NAME'})

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
app_numbers = application_df['APPLICATION_TYPE'].value_counts()
app_numbers

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace= list(app_numbers[app_numbers < 1000].index)

In [7]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_numbers = application_df["CLASSIFICATION"].value_counts()
classification_numbers

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C1248        1
C1820        1
C2190        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_numbers = classification_numbers[classification_numbers >= 1]
classification_numbers

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C1248        1
C1820        1
C2190        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_numbers[classification_numbers < 1800].index)

In [11]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")
    
# Check to make sure bucketing was successful
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df=pd.get_dummies(application_df)


In [13]:
# Split our preprocessed data into our features and target arrays
X=dummy_df.drop(columns={'IS_SUCCESSFUL'}, axis=1).values
y=dummy_df['IS_SUCCESSFUL'].values.reshape(-1, 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [17]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              period=5,
                              )

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=1000) 
#                    callbacks=[cp_callback])

Epoch 1/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5717 - accuracy: 0.7219
Epoch 2/1000
804/804 [==============================] - 1s 858us/step - loss: 0.5556 - accuracy: 0.7310
Epoch 3/1000
804/804 [==============================] - 1s 989us/step - loss: 0.5538 - accuracy: 0.7333
Epoch 4/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7332
Epoch 5/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7327
Epoch 6/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7344
Epoch 7/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5498 - accuracy: 0.7352
Epoch 8/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7355
Epoch 9/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7345
Epoch 10/1000
804/804 [==============================] - 1s 2ms/step - loss: 0

804/804 [==============================] - 1s 966us/step - loss: 0.5376 - accuracy: 0.7406
Epoch 81/1000
804/804 [==============================] - 1s 984us/step - loss: 0.5378 - accuracy: 0.7416
Epoch 82/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5379 - accuracy: 0.7401
Epoch 83/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5374 - accuracy: 0.7403
Epoch 84/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5371 - accuracy: 0.7402
Epoch 85/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5372 - accuracy: 0.7401
Epoch 86/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7405
Epoch 87/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7398
Epoch 88/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7411
Epoch 89/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5371

804/804 [==============================] - 1s 1ms/step - loss: 0.5363 - accuracy: 0.7419
Epoch 159/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7414
Epoch 160/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7420
Epoch 161/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5347 - accuracy: 0.7417
Epoch 162/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5345 - accuracy: 0.7418
Epoch 163/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5345 - accuracy: 0.7417
Epoch 164/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5348 - accuracy: 0.7418
Epoch 165/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7420
Epoch 166/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5346 - accuracy: 0.7415
Epoch 167/1000
804/804 [==============================] - 1s 1ms/step - loss: 0

804/804 [==============================] - 1s 980us/step - loss: 0.5343 - accuracy: 0.7408
Epoch 237/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5336 - accuracy: 0.7421
Epoch 238/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5341 - accuracy: 0.7418
Epoch 239/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5336 - accuracy: 0.7425
Epoch 240/1000
804/804 [==============================] - 2s 2ms/step - loss: 0.5336 - accuracy: 0.7421
Epoch 241/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5336 - accuracy: 0.7423
Epoch 242/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7411
Epoch 243/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5337 - accuracy: 0.7423
Epoch 244/1000
804/804 [==============================] - 1s 2ms/step - loss: 0.5336 - accuracy: 0.7420
Epoch 245/1000
804/804 [==============================] - 1s 1ms/step - loss:

KeyboardInterrupt: 

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("Alphabet_Soup_Charity.h5")